In [1]:
import joblib
from lightfm import LightFM
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from pathlib import Path
import kagglehub
import shutil
import os
from utils import BOOKS_DATA
import utils
from scipy.sparse import load_npz

/home/andrei/projects/main_projects/models-backend/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv(utils.BOOKS_DATA_BOOKS_PROCESSED)

In [3]:
y = load_npz(utils.BOOKS_DATA_Y)
y

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2805062 stored elements and shape (224512, 59134)>

In [4]:
df = pd.DataFrame.sparse.from_spmatrix(y)

In [5]:
r, c = y.nonzero()
data = np.full_like(r, 'Like', dtype='<U4')

In [6]:
df = pd.DataFrame({'book_id' : c, 'rating' : data})

In [7]:
b = pd.read_csv('books_processed.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'books_processed.csv'

In [8]:
import ast
import numpy as np
book_authors = b['authors'].apply(ast.literal_eval).reset_index()
def convert_book_authors(x):
    index, authors = x['index'], x['authors']
    authors = [[index] + [k, v] for k, v in authors.items()]
    df = pd.DataFrame(authors, columns=['book_id', 'author_id', 'role'])
    df['role'] = df['role'].replace({'': np.nan})
    return df
book_authors = pd.concat(book_authors.apply(convert_book_authors, axis=1).values, axis=0, ignore_index=True)

/tmp/ipykernel_5690/3736279845.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['role'] = df['role'].replace({'': np.nan})


In [9]:
book_categories = b['categories'].apply(ast.literal_eval).reset_index()
def convert_book_categories(x):
    index, categories = x['index'], x['categories']
    categories = [[index] + [v] for v in categories]
    df = pd.DataFrame(categories, columns=['book_id', 'category_id'])
    return df
book_categories = pd.concat(book_categories.apply(convert_book_categories, axis=1).values, axis=0, ignore_index=True)

In [28]:
from sklearn.preprocessing import LabelEncoder

categories = pd.read_csv('categories.csv')
categories.rename(columns={'0':'name'}, inplace=True)

In [37]:
lblEncoder = LabelEncoder().fit(categories['name'])

In [30]:
categories['id'] = lblEncoder.transform(categories['name'])

In [31]:
book_categories['category_id'] = lblEncoder.transform(book_categories['category_id'])

In [34]:
pd.read_csv('book_authors.csv', index_col='author_id')

,name
author_id,
0,Douglas Adams
1,Mark Watson
2,Delia Sherman
3,Anne McCaffrey
4,Mark Smylie
...,...
23616,Terri Delgado
23617,Alan Quah
23618,Matt Smith


In [50]:
pd.read_csv('categories.csv').value_counts()

0               
spooky              2
poverty             1
postmodern          1
post-apocalyptic    1
pornography         1
                   ..
fable               1
existentialism      1
ex-machina          1
ethics              1
fashion             1
Name: count, Length: 310, dtype: int64

In [39]:
book_authors

,book_id,author_id,role
0,0,12,Illustrations
1,1,1270,NaN
2,1,1269,NaN
3,1,4869,Original Novel
4,1,1271,Editor
...,...,...,...
148702,59131,18449,NaN
148703,59131,6138,Illustrator
148704,59131,6200,Illustrator
148705,59132,362,NaN


In [56]:
pd.read_csv('categories.csv')

,0
0,manga
1,comics
2,fantasy
3,fiction
4,horror
...,...
306,historical fiction
307,self-help
308,travel
309,shojo


In [8]:
model : LightFM = joblib.load(utils.BOOKS_DATA_MODEL)

(86983,)

In [13]:
model.get_user_representations()[0].shape

(224822,)

In [15]:
item_features = load_npz('books_data/item_features.npz')

In [19]:
model.get_item_representations(item_features)[0].shape

(59134,)